### Use of mip cmor tables with CMOR 3.7.3

To use the mip cmor tables with CMOR 3.7.3 a minor change to the structure of the MIP tables is needed. A forthcoming micro update to CMOR will correct the need for this (possibly CMOR version 3.7.4 or 3.8.0).

Note that you'll still need to either copy across or appropriately reference the CVs file for CMOR to work.

In the mean time the following procedure will create a 3.7.x compatible version of the tables


In [1]:
import json
import glob
import os
import shutil

In [2]:
# get list of tables files
tables = glob.glob('../Tables/*.json')

In [3]:
# files that are not mip tables
reference_files = ['../Auxillary_files/MIP_coordinate.json', '../Auxillary_files/MIP_formula_terms.json', 
                   '../Auxillary_files/MIP_grids.json']

In [4]:
# Set up output directory and clear it if has anything in it
output_directory = '../Tables_3.7.3'
if os.path.exists(output_directory):
    existing_files = glob.glob(output_directory + '/*')
    for filename in existing_files:
        if os.path.basename(filename).startswith('MIP_'):
            os.unlink(filename)
else:
    os.mkdir(output_directory)


In [5]:
# For each table
for table in tables:
    # skip reference files
    if table in reference_files:
        continue
        
    # read JSON
    with open(table, 'r') as fh:
        table_data = json.load(fh)

    # amend cmor version for compatibility check
    table_data['Header']['cmor_version'] = '3.7.3'

    # replace dimensions with space separated strings, 
    # and the same for the modeling_realm (otherwise 'realm' in the output files becomes "["
    for var_name, entry in table_data['variable_entry'].items():
        entry['dimensions'] = ' '.join(entry['dimensions'])
        entry['modeling_realm'] = ' '.join(entry['modeling_realm'])

    # write out to output_directory
    output_file = os.path.join(output_directory, 'MIP_{}.json'.format(table_data['Header']['table_id']))
    with open(output_file, 'w') as fh:
        json.dump(table_data, fh, sort_keys=True, indent=2)
    


In [6]:
# copy reference files
for filename in reference_files:
    destination = os.path.join(output_directory, os.path.basename(filename))
    shutil.copy(filename, destination)
